# Funny vs. Serious Single Sentence Explorer

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/davidbau/puns/blob/main/explore_funny_serious.ipynb)

**Research question:** Does an LLM represent "funny" and "serious" sentence endings differently, even when both sentences are grammatically valid and make sense?

## Dataset

We use the **funny_serious_150** dataset: 75 pairs of completed sentences where each pair has:
- A **serious** version: `"The dangerous iPhone was arrested and charged with assault."`
- A **funny** version: `"The dangerous iPhone was arrested and charged with battery."`

Both versions are valid English sentences. The only difference is the final word — one has the "pun" word, one has a straightforward word.

## Method

We collect activations from Llama-3.1-70B-Instruct at the **final period** of each sentence, then analyze whether the model's internal representation differs based on whether it just processed a funny or serious completion.

## Contents
1. Dataset preview
2. Collect activations (via NDIF)
3. Load data and compute separation metrics
4. Visualizations
5. Cross-validation / holdout analysis
6. Mean difference vs. Fisher LDA direction

In [ ]:
# Colab Setup (run this cell first if on Google Colab)

import sys, os, subprocess

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("Running on Google Colab - setting up environment...")

    # Clone the repository
    if not os.path.exists("puns"):
        subprocess.run(["git", "clone", "https://github.com/davidbau/puns.git"], check=True)
    os.chdir("puns")

    # Install dependencies - use nnsight from git for latest NDIF compatibility
    subprocess.run([sys.executable, "-m", "pip", "install", "-q", 
                    "git+https://github.com/ndif-team/nnsight.git",
                    "torch", "numpy", "matplotlib"], check=True)

    # Create output directories
    os.makedirs("results/raw_activations", exist_ok=True)

    # Note: nnsight will automatically read NDIF_API_KEY from Colab secrets
    # To set up: click the key icon in the left sidebar, add a secret named "NDIF_API_KEY"
    # Get your free API key at https://ndif.us/

    print("Setup complete!")
else:
    print("Running locally - no setup needed.")

In [ ]:
from pathlib import Path
import json
import numpy as np
from IPython.display import HTML, display
import matplotlib.pyplot as plt
%matplotlib inline

DATASET_FILE = Path("datasets/funny_serious_150.json")

---
## 1. Dataset Preview

Let's look at a few example prompt pairs to understand the data.

In [ ]:
with open(DATASET_FILE) as f:
    dataset = json.load(f)

print(f"Dataset: {len(dataset)} prompts ({len(dataset)//2} pairs)\n")
print("Sample prompt pairs:\n")
for i in range(0, min(10, len(dataset)), 2):
    serious = dataset[i]
    funny = dataset[i+1]
    print(f"Pair {serious['pair_id']}:")
    print(f"  Serious: \"{serious['prompt']}\"")
    print(f"  Funny:   \"{funny['prompt']}\"")
    print()

---
## 2. Collect Activations (via NDIF)

We collect hidden-state activations from Llama-3.1-70B-Instruct using [NDIF](https://ndif.us/) (remote GPU inference). This cell checks if data already exists; if not, it runs the collection with progress display.

**Note:** First-time collection requires an NDIF API key in `.env.local` and will run for several minutes.

In [ ]:
from collect_activations import ensure_activations

# This checks if data exists; if not, collects via NDIF with progress display
result = ensure_activations("funny_serious_150.json", position="pred_c")

META_FILE = result["meta_file"]
print(f"\nMeta file: {META_FILE}")

---
## 3. Load Data and Compute Separation Metrics

Now we load the activations and compute metrics to measure how well the model separates funny vs. serious sentences.

In [ ]:
# Data loading
from analyze_activations import load_activations, get_pair_indices

# Explicit-argument analysis functions (show exactly what data is needed)
from analyze_activations import (
    mean_difference,           # (X, is_group_a, is_group_b) → direction
    fisher_lda_direction,      # (X, is_group_a, is_group_b, shrinkage) → direction (covariance-corrected)
    compute_fisher_separation, # (X, is_group_a, is_group_b) → float
    compute_cohens_d,          # (X, is_group_a, is_group_b, direction) → float
)

# For visualization
from analyze_activations import contrastive_projection, holdout_analysis
from puns_viz import make_layer_viz

# Load activations: returns metadata, per-layer activations, and layer indices
meta, layer_data, layer_indices = load_activations(META_FILE)

# Extract the key arrays we need for analysis:
#   pair_ids:    which pair each prompt belongs to (0-74)
#   is_funny:    True for prompts with the pun completion  
#   is_serious:  True for prompts with the serious completion
pair_ids, is_funny, is_serious = get_pair_indices(meta)

print(f"Model: {meta['model']}")
print(f"Layers: {len(layer_indices)} (indices 0-{layer_indices[-1]})")
print(f"Hidden dimension: {meta['hidden_dim']}")
print(f"Total prompts: {len(is_funny)}")
print(f"  - Funny completions: {is_funny.sum()}")
print(f"  - Serious completions: {is_serious.sum()}")

### Separation metrics across layers

We'll compute two metrics at each layer:

**Fisher separation** = (distance between group means) / (average within-group spread)
- Measures how well-separated the two groups are in the full high-dimensional space
- Higher = better separation

**Cohen's d** = (mean_funny - mean_serious) / pooled_std, along the "contrastive direction"
- The contrastive direction is simply: mean(funny) - mean(serious), normalized
- Cohen's d tells us the effect size along this optimal separating direction
- Interpretation: 0.2 = small, 0.5 = medium, 0.8 = large, >1.0 = very large

In [ ]:
# Compute metrics at each layer
fisher_scores = []
cohens_d_scores = []

for layer_idx in layer_indices:
    # Get activations for this layer: shape (150, 8192)
    X = layer_data[layer_idx]
    
    # Fisher separation: uses the full activation vectors
    fisher = compute_fisher_separation(X, is_funny, is_serious)
    fisher_scores.append(fisher)
    
    # Contrastive direction: the normalized mean difference
    direction = mean_difference(X, is_funny, is_serious)
    
    # Cohen's d: effect size along the contrastive direction
    d = compute_cohens_d(X, is_funny, is_serious, direction)
    cohens_d_scores.append(d)

# Find peak layers
peak_fisher_idx = np.argmax(fisher_scores)
peak_cd_idx = np.argmax(cohens_d_scores)

print(f"Fisher separation peaks at layer {layer_indices[peak_fisher_idx]}: {fisher_scores[peak_fisher_idx]:.3f}")
print(f"Cohen's d peaks at layer {layer_indices[peak_cd_idx]}: {cohens_d_scores[peak_cd_idx]:.2f}")

In [ ]:
# Plot both metrics across layers
fig, ax1 = plt.subplots(figsize=(12, 4))

ax1.plot(layer_indices, fisher_scores, color='#E85D75', lw=2, label='Fisher separation')
ax1.set_xlabel('Layer', fontsize=11)
ax1.set_ylabel('Fisher separation', fontsize=11, color='#E85D75')
ax1.tick_params(axis='y', labelcolor='#E85D75')
ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)

ax2 = ax1.twinx()
ax2.plot(layer_indices, cohens_d_scores, color='#2EAD6B', lw=2, ls='--', label="Cohen's d")
ax2.set_ylabel("Cohen's d", fontsize=11, color='#2EAD6B')
ax2.tick_params(axis='y', labelcolor='#2EAD6B')
ax2.spines['top'].set_visible(False)

lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, fontsize=9, loc='upper left')
ax1.set_title('Funny vs. Serious: Separation by Layer', fontsize=13, fontweight='bold')
fig.tight_layout()
plt.show()

---
## 4. Visualizations

Let's look at the actual activations at the layer with highest Cohen's d.

The "contrastive projection" shows:
- **X-axis**: Projection onto the contrastive direction (mean_funny - mean_serious)
- **Y-axis**: First principal component of the residual (what's left after removing the contrastive direction)

Lines connect each pair (funny and serious versions of the same joke).

In [ ]:
# Get activations at peak Cohen's d layer
peak_layer = layer_indices[peak_cd_idx]
X_peak = layer_data[peak_layer]

# Compute 2D contrastive projection (uses meta internally for pair structure)
X_proj, components, var_ratios = contrastive_projection(X_peak, meta, n_components=2)

# Plot
fig, ax = plt.subplots(figsize=(10, 8))

# Draw lines connecting each pair
for pid in sorted(set(pair_ids)):
    mask = pair_ids == pid
    if mask.sum() == 2:
        pts = X_proj[mask]
        ax.plot(pts[:, 0], pts[:, 1], color='#888', alpha=0.4, lw=0.8, zorder=1)

# Scatter points
ax.scatter(X_proj[is_serious, 0], X_proj[is_serious, 1], c='#4A90D9', 
           s=40, alpha=0.7, label='Serious', edgecolors='white', lw=0.5, zorder=2)
ax.scatter(X_proj[is_funny, 0], X_proj[is_funny, 1], c='#E85D75',
           s=40, alpha=0.7, label='Funny', edgecolors='white', lw=0.5, zorder=2)

ax.set_xlabel(f'Contrastive direction ({var_ratios[0]:.1%} variance)', fontsize=11)
ax.set_ylabel(f'Residual PC1 ({var_ratios[1]:.1%} variance)', fontsize=11)
ax.set_title(f'Layer {peak_layer}: Contrastive Projection (Cohen\'s d = {cohens_d_scores[peak_cd_idx]:.2f})', 
             fontsize=13, fontweight='bold')
ax.legend(fontsize=10)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
fig.tight_layout()
plt.show()

In [ ]:
# 1D histogram: projection onto contrastive direction
direction = mean_difference(X_peak, is_funny, is_serious)
projections = X_peak @ direction

fig, ax = plt.subplots(figsize=(10, 4))
ax.hist(projections[is_serious], bins=15, alpha=0.6, color='#4A90D9', label='Serious', edgecolor='white')
ax.hist(projections[is_funny], bins=15, alpha=0.6, color='#E85D75', label='Funny', edgecolor='white')
ax.set_xlabel('Projection onto contrastive direction', fontsize=11)
ax.set_ylabel('Count', fontsize=11)
ax.set_title(f'Layer {peak_layer}: 1D Projections (Cohen\'s d = {cohens_d_scores[peak_cd_idx]:.2f})',
             fontsize=13, fontweight='bold')
ax.legend(fontsize=10)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
fig.tight_layout()
plt.show()

### Interactive 3D Layer Explorer

- **Drag** to rotate
- **Scroll/pinch** to zoom
- **Shift-drag** to pan
- **Layer slider** to move through all layers
- **Normalize layers** checkbox: scales each layer to the same magnitude (activations grow through layers)

In [ ]:
# Note: pred_file=False skips loading predictions because pun boost isn't meaningful
# for this dataset - these are completed sentences, not cloze predictions.
html = make_layer_viz(META_FILE, pred_file=False, width=900, height=600)
HTML(html)

---
## 5. Cross-Validation / Holdout Analysis

The contrastive direction is computed from the same data we use to measure Cohen's d. This could overfit — we might find a direction that separates *these specific* prompts but wouldn't generalize.

**Holdout analysis**: Split pairs into two groups, compute the contrastive direction from one group (training set), measure Cohen's d on the other (holdout set). If the cross-validated Cohen's d is close to the full-data Cohen's d, the separation is real (not overfit).

In [ ]:
# Run holdout analysis (splits pairs, trains direction on half, tests on other half)
holdout = holdout_analysis(layer_data, meta, n_splits=2, seed=42)

fig, ax = plt.subplots(figsize=(12, 4))
ax.plot(holdout['layer_indices'], holdout['cohens_d_full'], color='#2EAD6B', lw=2, label='Full data')
ax.plot(holdout['layer_indices'], holdout['cohens_d_cv'], color='#E85D75', lw=2, ls='--', label='Cross-validated')
ax.set_xlabel('Layer', fontsize=11)
ax.set_ylabel("Cohen's d", fontsize=11)
ax.set_title("Full Data vs. Cross-Validated Cohen's d", fontsize=13, fontweight='bold')
ax.legend(fontsize=10)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
fig.tight_layout()
plt.show()

# Report the gap at peak layer
cv_at_peak = holdout['cohens_d_cv'][peak_cd_idx]
full_at_peak = holdout['cohens_d_full'][peak_cd_idx]
print(f"\nAt peak layer {peak_layer}:")
print(f"  Full-data Cohen's d: {full_at_peak:.2f}")
print(f"  Cross-validated:     {cv_at_peak:.2f}")
print(f"  Ratio (CV/full):     {cv_at_peak/full_at_peak:.1%}")

### Visualizing Holdout: Training Direction Applied to Holdout Points

Let's visualize the holdout analysis at the peak layer. We compute the contrastive direction from the training set (even-numbered pairs) and project the holdout set (odd-numbered pairs) onto this direction.

In [ ]:
# Split pairs into train (even pair_ids) and holdout (odd pair_ids)
samples = meta["samples"]
unique_pairs = sorted(set(s["pair_id"] for s in samples))

train_mask = np.array([s["pair_id"] % 2 == 0 for s in samples])
holdout_mask = ~train_mask

X_train = X_peak[train_mask]
X_holdout = X_peak[holdout_mask]

is_funny_train = is_funny[train_mask]
is_serious_train = is_serious[train_mask]
is_funny_holdout = is_funny[holdout_mask]
is_serious_holdout = is_serious[holdout_mask]

# Compute direction from training set only
d_train = mean_difference(X_train, is_funny_train, is_serious_train)

# Project holdout points onto training direction
proj_holdout = X_holdout @ d_train

# Compute holdout Cohen's d
cd_holdout = compute_cohens_d(X_holdout, is_funny_holdout, is_serious_holdout, d_train)

# Plot
fig, ax = plt.subplots(figsize=(10, 4))
ax.hist(proj_holdout[is_serious_holdout], bins=12, alpha=0.6, color='#4A90D9', label='Serious (holdout)', edgecolor='white')
ax.hist(proj_holdout[is_funny_holdout], bins=12, alpha=0.6, color='#E85D75', label='Funny (holdout)', edgecolor='white')
ax.set_xlabel('Projection onto TRAINING contrastive direction', fontsize=11)
ax.set_ylabel('Count', fontsize=11)
ax.set_title(f'Layer {peak_layer}: Holdout Points on Training Direction (Cohen\'s d = {cd_holdout:.2f})',
             fontsize=13, fontweight='bold')
ax.legend(fontsize=10)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
fig.tight_layout()
plt.show()

print(f"Training set: {train_mask.sum()} samples ({train_mask.sum()//2} pairs)")
print(f"Holdout set:  {holdout_mask.sum()} samples ({holdout_mask.sum()//2} pairs)")
print(f"Holdout Cohen's d using training direction: {cd_holdout:.2f}")

### 3D Holdout Visualization

Interactive 3D view of holdout points, projected using the training direction at each layer.

In [ ]:
from puns_viz import make_holdout_viz

# Create 3D visualization of holdout points using training direction
html_holdout = make_holdout_viz(layer_data, meta, train_mask, width=900, height=600)
HTML(html_holdout)

---
## 6. Mean Difference vs. Fisher LDA Direction

The simple **mean difference** direction `d = mean(funny) - mean(serious)` doesn't account for within-class covariance. The **Fisher LDA direction** (Marks et al., 2024) uses the inverse within-class covariance to "whiten" the space:

```
d_lda = Σ_within⁻¹ @ (mean(funny) - mean(serious))
```

This can give a more robust direction when variance differs across dimensions. However, for high-dimensional data (8192 dims with only 150 samples), we need regularization (shrinkage toward identity) to make the covariance matrix invertible.

In [ ]:
# Compare mean difference vs LDA direction at peak layer
d_mean = mean_difference(X_peak, is_funny, is_serious)
d_lda = fisher_lda_direction(X_peak, is_funny, is_serious, shrinkage=0.1)

# Cohen's d along each direction
cd_mean = compute_cohens_d(X_peak, is_funny, is_serious, d_mean)
cd_lda = compute_cohens_d(X_peak, is_funny, is_serious, d_lda)

# Angle between the two directions
cos_angle = np.dot(d_mean, d_lda)
angle_deg = np.degrees(np.arccos(np.clip(np.abs(cos_angle), 0, 1)))

print(f"At layer {peak_layer}:")
print(f"  Mean difference Cohen's d: {cd_mean:.2f}")
print(f"  Fisher LDA Cohen's d:      {cd_lda:.2f}")
print(f"  Angle between directions:  {angle_deg:.1f}°")

In [ ]:
# Cross-validation comparison: does LDA generalize better?
holdout_mean = holdout_analysis(layer_data, meta, n_splits=2, seed=42, use_lda=False)
holdout_lda = holdout_analysis(layer_data, meta, n_splits=2, seed=42, use_lda=True, shrinkage=0.1)

fig, ax = plt.subplots(figsize=(12, 4))
ax.plot(holdout_mean['layer_indices'], holdout_mean['cohens_d_cv'], 
        color='#2EAD6B', lw=2, label='Mean difference (CV)')
ax.plot(holdout_lda['layer_indices'], holdout_lda['cohens_d_cv'], 
        color='#E85D75', lw=2, ls='--', label='Fisher LDA (CV)')
ax.set_xlabel('Layer', fontsize=11)
ax.set_ylabel("Cohen's d (cross-validated)", fontsize=11)
ax.set_title("Cross-Validated Cohen's d: Mean Difference vs. Fisher LDA", fontsize=13, fontweight='bold')
ax.legend(fontsize=10)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
fig.tight_layout()
plt.show()

# Report at peak layer
cv_mean_at_peak = holdout_mean['cohens_d_cv'][peak_cd_idx]
cv_lda_at_peak = holdout_lda['cohens_d_cv'][peak_cd_idx]
print(f"\nCross-validated at layer {peak_layer}:")
print(f"  Mean difference: {cv_mean_at_peak:.2f}")
print(f"  Fisher LDA:      {cv_lda_at_peak:.2f}")